In [34]:
import pandas as pd
import os

In [35]:
import csv

def parse_txt_to_csv(txt_file, csv_file):
    with open(txt_file, 'r') as in_file:
        stripped = (line.strip() for line in in_file)
        lines = (line.split("|")[1:-1] for line in stripped if line)  # Exclude first and last empty strings
        cleaned_lines = ([item.strip() for item in line] for line in lines)
        with open(csv_file, 'w', newline='') as out_file:
            writer = csv.writer(out_file)
            writer.writerow(('ID', 'Timestamp', 'Value'))  # writing headers
            writer.writerows(cleaned_lines)  # writing content

# Call the function like this:
parse_txt_to_csv('../data/original/building/temperature.txt', '../data/original/building/temperature.csv')

In [36]:
data = pd.read_csv('../data/original/building/temperature.csv', index_col='ID')

In [37]:
data

,Timestamp,Value
ID,,
1000407958,2023-05-02 09:00:00,20.823185
1000407959,2023-05-02 09:01:00,20.829910
1000407960,2023-05-02 09:02:00,20.813765
1000407961,2023-05-02 09:03:00,20.825876
1000407962,2023-05-02 09:04:00,20.813765
...,...,...
1000438624,2023-05-23 15:56:00,20.910439
1000438625,2023-05-23 15:57:00,20.921150
1000438626,2023-05-23 15:58:00,20.937208


In [19]:
def jsons_to_csv(directory, csv_file):
    data_frames = []  # list to store data frames

    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            df = pd.read_json(file_path)  # directly read json file into dataframe
            data_frames.append(df)  # add dataframe to list

    # Concatenate all dataframes, and write to CSV
    final_df = pd.concat(data_frames, ignore_index=True)
    final_df.to_csv(csv_file, index=False)

# Call the function like this:
jsons_to_csv('../data/original/building/occupation/', '../data/original/building/occupation.csv')

In [38]:
mapping = pd.read_csv('../data/original/building/place_ids.csv', delimiter=';')

In [39]:
occupancy = pd.read_csv('../data/original/building/occupation.csv')

In [40]:
merge = pd.merge(occupancy, mapping, left_on='asset', right_on='id', how='left' )

In [41]:
def add_place_id_column(df):
    place_id = []
    for name in df['name']:
        if name.startswith('0.B'):
            place_id.append(1)
        elif name.startswith('1.D'):
            place_id.append(3)
        elif name.startswith('L1.'):
            place_id.append(4)
        else:
            place_id.append('')
    df['place_id'] = place_id
    return df

merge = add_place_id_column(merge)

In [42]:
def add_atrium_column(df):
    atrium = []
    for name in df['name']:
        if name.startswith('0.B') or name.startswith('1.D') or name.startswith('L1.') or name.startswith('1.C'):
            atrium.append(True)
        else:
            atrium.append(False)
    df['atrium'] = atrium
    return df

merge = add_atrium_column(merge)

In [32]:
merge['name'].unique()


array(['0.A01', '0.A02', '0.A03', '0.A04', '0.A05', '0.A06', '0.A07',
       '0.A08', '0.A09', '0.A10', '0.A11', '0.A12', '0.A13', '0.A14',
       '0.A15', '0.A16', '1.A01', '1.A02', '1.A03', '1.A04', '1.A05',
       '1.A06', '1.A07', '1.A08', '1.A09', '1.A10', '1.A11', '1.A12',
       '1.A13', '1.A14', '1.A15', '1.A16', '1.A17', '1.A18', '1.A19',
       '1.A20', '1.A21', '1.A22', '1.A23', '1.A24', '1.A25', '1.A26',
       '1.B01', '1.B02', '1.B03', '1.B04', '1.B07', '1.B08', '1.B09',
       '1.B10', '1.B11', '1.B12', '1.B13', '1.B14', '1.C01', '1.C02',
       '1.C03', '1.C04', '1.C05', '1.C06', '1.C07', '1.C08', '1.C09',
       '1.C10', '1.C11', '1.C12', '1.C13', '1.C14', '1.C15', '1.C16',
       '1.C17', '1.C18', '1.C19', '1.C20', '1.C21', '1.C22', '1.C23',
       '1.C24', '1.D01', '1.D02', '1.D03', '1.D04', '1.D05', '1.D06',
       '1.D07', '1.D08', '1.D09', '1.D10', '1.D11', '1.D12', '1.D13',
       '1.D14', '1.D15', '1.D16', '2.A01', '2.A02', '2.A03', '2.A04',
       '2.A05', '2.A

In [43]:

merge['utc_time'] = pd.to_datetime(merge['utc_time'])
merge['utc_time'] = merge['utc_time'].dt.tz_convert('Europe/Amsterdam')
merge['utc_time'] = merge['utc_time'].dt.tz_localize(None)

In [44]:
merge.rename(columns={'utc_time': 'timestamp'}, inplace=True)

In [45]:
merge

,asset,timestamp,occupied,id,name,place_id,atrium
0,2692,2023-05-05 00:00:00,False,2692,0.A01,,False
1,2692,2023-05-05 00:05:00,False,2692,0.A01,,False
2,2692,2023-05-05 00:10:00,False,2692,0.A01,,False
3,2692,2023-05-05 00:15:00,False,2692,0.A01,,False
4,2692,2023-05-05 00:20:00,False,2692,0.A01,,False
...,...,...,...,...,...,...,...
1159483,4149,2023-05-17 23:35:00,False,4149,L1.E07,4,True
1159484,4149,2023-05-17 23:40:00,False,4149,L1.E07,4,True
1159485,4149,2023-05-17 23:45:00,False,4149,L1.E07,4,True
1159486,4149,2023-05-17 23:50:00,False,4149,L1.E07,4,True


In [28]:
merge.to_csv('../data/processed/building/occupation.csv', index=False)